In [ ]:
!pip install tensorflow numpy matplotlib datasets pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.9/274.9 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.0/129.0 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.1/231.1 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.1/344.1 kB 21.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.12.0
    Uninstalling fsspec-2024.12.0:
 

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential, layers, Input
from tensorflow.keras.layers import (
    Conv2D, BatchNormalization, MaxPooling2D, Activation, Flatten, Dense, Dropout,
    RandomFlip, RandomRotation, RandomZoom, RandomContrast, RandomBrightness
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from datasets import load_dataset

In [ ]:
# Set mixed precision policy
tf.keras.mixed_precision.set_global_policy('mixed_bfloat16')

In [ ]:
# Initialize TPU
resolver = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
tpu_strategy = tf.distribute.TPUStrategy(resolver)
print("TPU Devices:", tf.config.list_logical_devices('TPU'))

TPU Devices: [LogicalDevice(name='/device:TPU:0', device_type='TPU'), LogicalDevice(name='/device:TPU:1', device_type='TPU'), LogicalDevice(name='/device:TPU:2', device_type='TPU'), LogicalDevice(name='/device:TPU:3', device_type='TPU'), LogicalDevice(name='/device:TPU:4', device_type='TPU'), LogicalDevice(name='/device:TPU:5', device_type='TPU'), LogicalDevice(name='/device:TPU:6', device_type='TPU'), LogicalDevice(name='/device:TPU:7', device_type='TPU')]


In [ ]:
# Load dataset
ds = load_dataset("JamieWithofs/Deepfake-and-real-images-4")
ds

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/622 [00:00<?, ?B/s]

train-00000-of-00006.parquet:   0%|          | 0.00/481M [00:00<?, ?B/s]

train-00001-of-00006.parquet:   0%|          | 0.00/207M [00:00<?, ?B/s]

train-00002-of-00006.parquet:   0%|          | 0.00/287M [00:00<?, ?B/s]

...

validation-00001-of-00002.parquet:   0%|          | 0.00/385M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/121159 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/35304 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/53184 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 121159
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 35304
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 53184
    })
})

In [ ]:
IMG_SIZE = 256
BATCH_SIZE = 64 * tpu_strategy.num_replicas_in_sync

In [ ]:
# Creating augmentation layers
flip_layer = RandomFlip("horizontal")
rotation_layer = RandomRotation(0.1)
zoom_layer = RandomZoom(0.1)
contrast_layer = RandomContrast(0.1)
brightness_layer = RandomBrightness(0.2)

In [ ]:
def preprocess(example):
    image = tf.image.resize(example['image'], [IMG_SIZE, IMG_SIZE])
    image = tf.cast(image, tf.float32) / 255.0

    def aug():
        idx = tf.random.uniform([], minval=0, maxval=5, dtype=tf.int32)
        augmented = tf.switch_case(idx, branch_fns={
            0: lambda: flip_layer(image),
            1: lambda: rotation_layer(image),
            2: lambda: zoom_layer(image),
            3: lambda: contrast_layer(image),
            4: lambda: brightness_layer(image)
        })
        return augmented

    # With a 50% chance, apply one random augmentation
    image = tf.cond(
        tf.random.uniform(()) > 0.5,
        aug,
        lambda: tf.cast(image, tf.bfloat16)
    )

    return image, example['label']

In [ ]:
# Convert Hugging Face dataset to TensorFlow datasets with on-the-fly preprocessing
def to_tf_dataset(hf_dataset):
    return hf_dataset.to_tf_dataset(
        columns=['image', 'label'],
        shuffle=True if hf_dataset == ds['train'] else False,
        batch_size=BATCH_SIZE,
    ).map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
# Create TensorFlow datasets
train_ds = to_tf_dataset(ds['train'])
val_ds = to_tf_dataset(ds['validation'])
test_ds = to_tf_dataset(ds['test'])

In [ ]:
# Cache and prefetch for better performance
train_ds = train_ds.prefetch(tf.data.AUTOTUNE).cache()
val_ds = val_ds.prefetch(tf.data.AUTOTUNE).cache()
test_ds = test_ds.prefetch(tf.data.AUTOTUNE)

In [ ]:
# Learning rate schedule
class WarmUpExponentialDecay(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, initial_lr, decay_steps, decay_rate, warmup_steps):
        super().__init__()
        self.initial_lr = initial_lr
        self.decay_steps = decay_steps
        self.decay_rate = decay_rate
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        step = tf.cast(step, tf.float32)
        warmup_steps = tf.cast(self.warmup_steps, tf.float32)
        decay_steps = tf.cast(self.decay_steps, tf.float32)
        initial_lr = tf.cast(self.initial_lr, tf.float32)
        decay_rate = tf.cast(self.decay_rate, tf.float32)

        warmup_lr = initial_lr * (step / warmup_steps)
        decay_lr = initial_lr * tf.math.exp(-decay_rate * ((step - warmup_steps) / decay_steps))
        return tf.cond(step < warmup_steps, lambda: warmup_lr, lambda: decay_lr)

    def get_config(self):
        return {
            "initial_lr": self.initial_lr,
            "decay_steps": self.decay_steps,
            "decay_rate": self.decay_rate,
            "warmup_steps": self.warmup_steps
        }

In [ ]:
initial_lr = 1e-3
decay_steps = 1000
decay_rate = 0.1
warmup_steps = 500

lr_schedule = WarmUpExponentialDecay(initial_lr, decay_steps, decay_rate, warmup_steps)

In [ ]:
with tpu_strategy.scope():
    model = Sequential([
        Input(shape=(IMG_SIZE, IMG_SIZE, 3), dtype=tf.bfloat16),

        Conv2D(32, (3, 3), padding='same'),
        BatchNormalization(dtype=tf.float32),
        Activation('relu'),
        MaxPooling2D((2, 2)),

        Conv2D(64, (3, 3), padding='same'),
        BatchNormalization(dtype=tf.float32),
        Activation('relu'),
        MaxPooling2D((2, 2)),

        Conv2D(128, (3, 3), padding='same'),
        BatchNormalization(dtype=tf.float32),
        Activation('relu'),
        MaxPooling2D((2, 2)),

        Conv2D(256, (3, 3), padding='same'),
        BatchNormalization(dtype=tf.float32),
        Activation('relu'),
        MaxPooling2D((2, 2)),

        Conv2D(512, (3, 3), padding='same'),
        BatchNormalization(dtype=tf.float32),
        Activation('relu'),
        MaxPooling2D((2, 2)),

        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid', dtype=tf.float32)
    ])

    model.compile(
        optimizer=Adam(learning_rate=lr_schedule),
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
    )

In [ ]:
# Callbacks for early stopping and model checkpointing
callbacks = [
    EarlyStopping(patience=10, restore_best_weights=True),
    ModelCheckpoint('best_model.keras', save_best_only=True)
]

In [ ]:
# Train the model
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=30,
    callbacks=callbacks
)

Epoch 1/30
237/237 ━━━━━━━━━━━━━━━━━━━━ 643s 3s/step - accuracy: 0.6063 - auc: 0.6406 - loss: 0.7226 - val_accuracy: 0.5072 - val_auc: 0.6258 - val_loss: 0.7812
Epoch 2/30
237/237 ━━━━━━━━━━━━━━━━━━━━ 97s 406ms/step - accuracy: 0.7592 - auc: 0.8467 - loss: 0.4743 - val_accuracy: 0.5639 - val_auc: 0.6726 - val_loss: 0.6717
Epoch 3/30
237/237 ━━━━━━━━━━━━━━━━━━━━ 96s 405ms/step - accuracy: 0.8013 - auc: 0.8973 - loss: 0.4007 - val_accuracy: 0.5135 - val_auc: 0.7294 - val_loss: 0.6868
Epoch 4/30
237/237 ━━━━━━━━━━━━━━━━━━━━ 98s 411ms/step - accuracy: 0.8419 - auc: 0.9314 - loss: 0.3280 - val_accuracy: 0.7004 - val_auc: 0.8028 - val_loss: 0.5895
Epoch 5/30
237/237 ━━━━━━━━━━━━━━━━━━━━ 96s 403ms/step - accuracy: 0.8577 - auc: 0.9464 - loss: 0.2884 - val_accuracy: 0.5340 - val_auc: 0.7770 - val_loss: 0.6148
Epoch 6/30
237/237 ━━━━━━━━━━━━━━━━━━━━ 97s 407ms/step - accuracy: 0.9003 - auc: 0.9688 - loss: 0.2200 - val_accuracy: 0.6996 - val_auc: 0.8045 - val_loss: 0.5888
Epoch 7/30
237/237 ━━━━━

In [ ]:
# Evaluate the model
print("\nTesting Results:")
test_loss, test_acc, test_auc = model.evaluate(test_ds)
print(f"Test Accuracy: {test_acc:.4f}")
print(f"Test AUC: {test_auc:.4f}")
print(f"Test Loss: {test_loss:.4f}")


Testing Results:
69/69 ━━━━━━━━━━━━━━━━━━━━ 122s 2s/step - accuracy: 0.7856 - auc: 0.4542 - loss: 0.4166
Test Accuracy: 0.7902
Test AUC: 0.8888
Test Loss: 0.4261
